# Brazilian Newspaper analysis

In this project, we'll use a dataset from a Brazilian Newspaper called "Folha de São Paulo".

We're gonna use word embeddings, tensorboard and rnn's and check what we can find in this dataset.

You can find the dataset at [kaggle](https://www.kaggle.com/marlesson/news-of-the-site-folhauol).

I want to find in this study case:

+ Political opinions
+ Check if this newspaper is impartial or biased

So, let's start it

## Loading the data

After downloading the dataset, put it on a directory `data/` and let's load it using pandas.

In [1]:
# Import dependencies

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import os
import pickle

In [2]:
dataset = pd.read_csv('data/articles.csv')

dataset.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


## Preprocessing the data

### Merging title and text
To maintain article titles and text related, let's merge then together and use this merged text as our inputs

In [3]:
# Merges the title and text with a separator (---)
merged_text = [str(title) + ' ---- ' + str(text) for title, text in zip(dataset.title, dataset.text)]

print(merged_text[0])

Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral ---- Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está "lascado", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer.  Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência.  "Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas", disse Lula durante um seminário sobre educação em Brasília. "Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não 

### Tokenizing punctuation
We need to tokenize all text punctuation, otherwise the network will see punctuated words differently (eg: hello != hello!)

In [4]:
def token_lookup():
    tokens = {
        '.'  : 'period',
        ','  : 'comma',
        '"'  : 'quote',
        '\'' : 'single_quote',
        ';'  : 'semicolon',
        ':'  : 'colon',
        '!'  : 'exclamation_mark',
        '?'  : 'question_mark',
        '('  : 'parentheses_left',
        ')'  : 'parentheses_right',
        '--' : 'dash',
        '\n' : 'return'
    }
    
    return {token: '||{0}||'.format(value) for token, value in tokens.items()}

token_dict = token_lookup()

tokenized_text = []

for text in merged_text:
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))
    
    tokenized_text.append(text)

print(tokenized_text[0])

Lula diz que está  ||single_quote|| lascado ||single_quote||  ||comma||  mas que ainda tem força como cabo eleitoral  ||dash||  ||dash||  Com a possibilidade de uma condenação impedir sua candidatura em 2018 ||comma||  o ex-presidente Luiz Inácio Lula da Silva fez ||comma||  nesta segunda  ||parentheses_left|| 9 ||parentheses_right||  ||comma||  um discurso inflamado contra a Lava Jato ||comma||  no qual disse saber que está  ||quote|| lascado ||quote||  ||comma||  exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que ||comma||  mesmo fora da disputa pelo Planalto ||comma||  será um cabo eleitoral expressivo para a sucessão de Michel Temer ||period||   Segundo o petista ||comma||  réu em sete ações penais ||comma||  o objetivo de Moro é impedir sua candidatura no ano que vem ||comma||  desidratando-o ||comma||  inclusive ||comma||  no apoio a um nome alternativo ||comma||  como o do ex-prefeito de São Paulo Fernando Haddad  ||parentheses_left|| PT ||parentheses_right||  ||com

### Lookup tables

We need to create two dicts: `word_to_int` and `int_to_word`.

In [8]:
def lookup_tables(tokenized_text):
    vocab = set()
    
    for text in tokenized_text:
        vocab = vocab.union(set(text.split()))
    
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = {ii: word for ii, word in enumerate(vocab)}
    
    return vocab_to_int, int_to_vocab

vocab_to_int, int_to_vocab = lookup_tables(tokenized_text)

print('First ten vocab words: ')
print(list(vocab_to_int.items())[0:10])
print('\nVocab length:')
print(len(vocab_to_int))

pickle.dump((tokenized_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess/preprocess.p', 'wb'))

First ten vocab words: 
[('orginária', 0), ('Lombok', 1), ('Mercearias', 2), ('compartilhando', 3), ('estadocentrismo', 4), ('férreas', 5), ('rikênio', 6), ('CAMINHÕES-PIPA', 7), ('Eindrücke', 8), ('grastronômicas', 9)]

Vocab length:
562119
